<a href="https://colab.research.google.com/github/Bristopher/ICloud-Photos-to-Google-Drive/blob/main/Icloud_to_GDrive_v1_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First import your Google Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


Run this script to append "\~Starred\~" at beginning of filename for extra content to folder (like favorites / to organize) 

    *Default download folder is "Skate/Skate Clips", there's a good chance this isn't what you want. Go down to the 'variable' ascii art's to change this

In [ ]:
## Added functionality for a single unzipped file, next todo: fix variable renaming mess, updating comments
import os, zipfile, urllib.request, shutil
from urllib.parse import urlparse
from pathlib import Path


###SharedBy = input('On icloud page after clicking download what text is under "Check your downloads folder for:"? ("Check your downloads folder for: iCloud Photos from Christopher Buzicky", enter "iCloud Photos from Christopher Buzicky") ');
ClipsFolderName = input("Enter name for clips with date (2022 1-3-22 Chris kickflips) ");
ClipsFolderNameSub = input("Enter 'stard' to append ~Starred~ before each file or 86 (to export file in different parent folder for seperate share options) enter anything else to name create and export to subfolder (just hit enter for normal operation) ");
FileToDl = input('Paste link from Network tab in DevConsole ');
###SharedByFullVarName = SharedBy + "/.";
ClipsFolderName = ClipsFolderName.replace("/", "-");
TmpExtractedPath = '/content/IcloudDl/Extracted/'


################################################################################################################################################################################################################################################### 
# __     __         _       _     _      #
# \ \   / /_ _ _ __(_) __ _| |__ | | ___ #
#  \ \ / / _` | '__| |/ _` | '_ \| |/ _ \#
#   \ V / (_| | |  | | (_| | |_) | |  __/#
#    \_/ \__,_|_|  |_|\__,_|_.__/|_|\___|#
#                                        #Change 'Skate Clips' to whatever folder you want to download to in your Google Drive
OutputToGDrive = '/gdrive/MyDrive/Skate/Skate Clips/' + ClipsFolderName
OutputToGDrive86 = r'/gdrive/MyDrive/Skate/Skate Clips ~Redacted~/' + ClipsFolderName
###################################################################################################################################################################################################################################################

#Creates local  workdir if it doesn't exist (exists if you want to rerun script in same session)
mypath = ('/content/IcloudDl');
mypathtwo = ('/content/IcloudDl/Extracted');
if not os.path.isdir(mypath):
   os.makedirs(mypath);
   os.makedirs(mypathtwo);

#Creates Directory in your Google Drive if it doesn't exist
if (ClipsFolderNameSub == "86"):
  OutputToGDrive = OutputToGDrive86

if not os.path.isdir(OutputToGDrive):
  os.makedirs(OutputToGDrive);

DlUrlDirtyFileName = urlparse(FileToDl).path
DlUrlCleanFileName = DlUrlDirtyFileName.split('/')[-1]
DlUrlFileNameExtension = os.path.splitext(DlUrlCleanFileName)[-1]

if (DlUrlFileNameExtension.casefold() == '.jpeg' or DlUrlFileNameExtension.casefold() == '.jpg' or DlUrlFileNameExtension.casefold() == '.mp4' or DlUrlFileNameExtension.casefold() == '.mov'):
  DlUrlSingleFilePath =  os.path.join(TmpExtractedPath, DlUrlCleanFileName)
  urllib.request.urlretrieve(FileToDl, DlUrlSingleFilePath);
  UnZippedFile = DlUrlSingleFilePath
  isZipFile = False;
    

elif(DlUrlFileNameExtension.casefold() == '.zip'):
  #Downloads ICloud Share Link
  urllib.request.urlretrieve(FileToDl, '/content/IcloudDl/iCloud Photos.zip');
  #Unzip's Downloaded ICloud 
  if(zipfile.is_zipfile('/content/IcloudDl/iCloud Photos')):
    with zipfile.ZipFile('/content/IcloudDl/iCloud Photos.zip', 'r') as zip_ref:
        zip_ref.extractall(TmpExtractedPath)
        isZipFile = True;    
  #Appends name of Unzipped Folder to get full path
  UnZippedFile = r'/content/IcloudDl/Extracted/'
  UnZippedFileFolderName = [x for x in os.listdir(UnZippedFile) if os.path.isdir(os.path.join(UnZippedFile,x))] #Source: https://stackoverflow.com/questions/16455244/assign-a-variable-to-every-directory-inside-one-folder-python
  UnZippedFileNoDot = UnZippedFile + UnZippedFileFolderName[0]
  if(isZipFile):
    UnZippedFile = UnZippedFile + UnZippedFileFolderName[0] + "/."; 

################################################################################################################################################################################################################################################### 
# __     __         _       _     _      #
# \ \   / /_ _ _ __(_) __ _| |__ | | ___ #
#  \ \ / / _` | '__| |/ _` | '_ \| |/ _ \#
#   \ V / (_| | |  | | (_| | |_) | |  __/#
#    \_/ \__,_|_|  |_|\__,_|_.__/|_|\___|#
#                                        #Change "stard" for a different keyword, change "~Starred~ " to append a different text to each file
#Checks if user wants a sub folder, if they do it appends the folder name they chose
ClipsFolderNameSub = ClipsFolderNameSub.replace("/", "-");
if(ClipsFolderNameSub != ""):
  if(ClipsFolderNameSub == "stard"):
    #code here to append '~Starred~ '
    if(os.path.isdir(UnZippedFile)):
      ListofUnZippedFiles = [x for x in os.listdir(UnZippedFile) if os.path.isfile(os.path.join(UnZippedFile,x))] #Source: https://stackoverflow.com/questions/16455244/assign-a-variable-to-every-directory-inside-one-folder-python
      for x in ListofUnZippedFiles:
        old_file = os.path.join(UnZippedFileNoDot, x)
        y = r"~Starred~ " + x
        new_file = os.path.join(UnZippedFileNoDot, y)

        os.replace(old_file, new_file)
    
    else:
      OriginalUnZippedFileName = UnZippedFile
      DlUrlCleanFileName = r"~Starred~ " + DlUrlCleanFileName
      UnZippedFile =  r'/content/IcloudDl/Extracted/' + DlUrlCleanFileName
      os.replace(OriginalUnZippedFileName, UnZippedFile)
  
  else:
    ClipsFolderName = ClipsFolderName + "/" + ClipsFolderNameSub
###################################################################################################################################################################################################################################################



#Finally copys unzipped content to GDrive
!cp -a "$UnZippedFile" "$OutputToGDrive"

#If you want to rerun in the same session this gets rid of the possiblity of duplicates
if os.path.exists('/content/IcloudDl'):
   shutil.rmtree(mypath)